In [244]:
import pandas as pd
#!pip install xgboost
#from __future__ import division
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost
import math
import seaborn as sns
import sklearn
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
from sklearn import tree, linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import r2_score
from scipy import stats
from scipy.special import boxcox, inv_boxcox
from sklearn.model_selection import StratifiedKFold
from scipy.stats import norm
import numpy as np 
from sklearn.feature_selection import RFECV

In [330]:
data=pd.read_csv('file4.csv')

In [331]:
data.head()

,Unnamed: 0,Customer,Customer Lifetime Value,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Total Claim Amount,...,category_Oregon_Rural,category_Oregon_Suburban,category_Oregon_Urban,category_Washington_Rural,category_Washington_Suburban,category_Washington_Urban,category_Agent,category_Branch,category_Call Center,category_Web
0,0,BU79786,2763.519279,56274,69,32,5,0,1,384.811147,...,0,0,0,0,1,0,1,0,0,0
1,1,QZ44356,6979.535903,0,94,13,42,0,8,1131.464935,...,0,0,0,0,0,0,1,0,0,0
2,2,AI49188,12887.431650,48767,108,18,38,0,2,566.472247,...,0,0,0,0,0,0,1,0,0,0
3,3,WW63253,7645.861827,0,106,18,65,0,7,529.881344,...,0,0,0,0,0,0,0,0,1,0
4,4,HB64268,2813.692575,43836,73,12,44,0,1,138.130879,...,0,0,0,1,0,0,1,0,0,0


In [332]:
data=data.drop(['Unnamed: 0'],axis=1)

In [333]:
data.columns

Index(['Customer', 'Customer Lifetime Value', 'Income', 'Monthly Premium Auto',
       'Months Since Last Claim', 'Months Since Policy Inception',
       'Number of Open Complaints', 'Number of Policies', 'Total Claim Amount',
       'CLV Compl', 'category_Basic', 'category_Extended', 'category_Premium',
       'category_Bachelor', 'category_College', 'category_Doctor',
       'category_High School or Below', 'category_Master', 'category_Disabled',
       'category_Employed', 'category_Medical Leave', 'category_Retired',
       'category_Unemployed', 'category_Divorced', 'category_Married',
       'category_Single', 'category_Corporate L1', 'category_Corporate L2',
       'category_Corporate L3', 'category_Personal L1', 'category_Personal L2',
       'category_Personal L3', 'category_Special L1', 'category_Special L2',
       'category_Special L3', 'category_Four-Door Car_Vehicle Size',
       'category_Luxury Car_Vehicle Size', 'category_Luxury SUV_Vehicle Size',
       'category_SUV_

In [334]:
new_data = data[data.columns]
new_data=new_data.drop(['Customer Lifetime Value'],axis=1)
X = new_data
y = data['Customer Lifetime Value']
y.shape

(9134,)

In [335]:
p=X
p1=X['Customer']
p=p.drop(['Customer'],axis=1)
p2=y

In [336]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y ,test_size=0.02)
X_train.head()

,Customer,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Total Claim Amount,CLV Compl,category_Basic,...,category_Oregon_Rural,category_Oregon_Suburban,category_Oregon_Urban,category_Washington_Rural,category_Washington_Suburban,category_Washington_Urban,category_Agent,category_Branch,category_Call Center,category_Web
5025,QE73187,66152,111,14,18,0,1,532.800000,1373.377251,1,...,0,1,0,0,0,0,0,0,1,0
2527,RD61361,0,84,0,76,0,4,856.347474,20344.632490,0,...,0,0,0,0,0,0,0,1,0,0
5589,PK20739,25074,112,17,8,0,2,537.600000,3104.333537,0,...,0,1,0,0,0,0,0,1,0,0
4848,ZW93288,22705,239,19,59,0,7,1294.700423,84001.847029,0,...,0,0,0,0,0,0,1,0,0,0
6582,ZL82075,56005,68,31,30,1,3,326.400000,5518.363193,1,...,0,0,0,0,0,0,0,1,0,0


In [337]:
train_data,fitted_lambda = stats.boxcox(X_train['Monthly Premium Auto'])
X_train['Monthly Premium Auto'] = train_data
# use lambda value to transform test data
test_data = stats.boxcox(X_test['Monthly Premium Auto'], fitted_lambda)
X_test['Monthly Premium Auto']=test_data

/home/ayantika/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ayantika/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [338]:
y_train_new= np.log(y_train)
X_train=X_train.drop(['Customer'],axis=1)
y=X_test['Customer']
X_test=X_test.drop(['Customer'],axis=1)

In [339]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0) 
regressor.fit(X_train, y_train_new) 
y_pred = regressor.predict(X_test)
r2_score(y_test,np.exp(y_pred))

0.7757258715062711

In [343]:
#on training Set
predictions_rf = np.exp(regressor.predict(X_train))
print("R2 Score :",r2_score(y_train, predictions_rf))


R2 Score : 0.9442085994618442


In [340]:
#On cross Validation Set
xgb = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.7,
                           colsample_bytree=1, max_depth=7)
xgb.fit(X_train,y_train_new)
predictions = np.exp(xgb.predict(X_test))
print("Explained_variance_score :",explained_variance_score(predictions,y_test))
print("R2 Score :",r2_score(y_test, predictions))


/home/ayantika/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[20:09:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Explained_variance_score : 0.6221962061341628
R2 Score : 0.7530391300954867


In [383]:
#on training Set
predictions1 = np.exp(xgb.predict(X_train))
print("Explained_variance_score :",explained_variance_score(predictions1,y_train))
print("R2 Score :",r2_score(y_train, predictions1))


Explained_variance_score : 0.7217844618924285
R2 Score : 0.8250428647141711


In [384]:
df_rf = pd.DataFrame(y_test) 

In [404]:
df_rf['Predicted_xgb']=np.exp(y_pred)
df_rf.to_csv('df_rf.csv') 

In [386]:
df_rf.head()

,Customer Lifetime Value,Predicted_xgb
3261,2807.049427,2806.613245
5319,5657.685223,5575.640460
8656,36632.534320,24123.900635
9023,2117.989147,2217.818367
2971,3086.013544,3023.651137


In [405]:
df_rf_train=pd.DataFrame(y_train)
df_rf_train['Predicted_xgb']=predictions_rf
df_rf_train.to_csv('df_rf_train.csv') 

In [388]:
df_rf_train.head()

,Customer Lifetime Value,Predicted_xgb
5025,4407.973120,4421.094218
2527,6102.953572,6058.068000
5589,13248.637350,14262.130609
4848,19160.989940,19160.989940
6582,5479.555081,5475.417134


In [389]:
result=pd.concat([df_rf,df_rf_train])
result.to_csv('results.csv') 

In [390]:
def mean_absolute_percentage_error(y_test, predictions): 
    y_true, y_pred = np.array(y_test), np.array(predictions)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [391]:
print("mean_absolute_percentage_error on CrossValidation set(Random Forest)",mean_absolute_percentage_error(y_test,np.exp(y_pred)))
print("mean_absolute_percentage_error on Training set(Random Forest)",mean_absolute_percentage_error(y_train, predictions_rf))
print("mean_absolute_percentage_error on CrossValidation set(XGBoost)",mean_absolute_percentage_error(y_test, predictions))
print("mean_absolute_percentage_error on Training set(XGBoost)",mean_absolute_percentage_error(y_train, predictions1))


mean_absolute_percentage_error on CrossValidation set(Random Forest) 8.592100911317885
mean_absolute_percentage_error on Training set(Random Forest) 3.3494212958850986
mean_absolute_percentage_error on CrossValidation set(XGBoost) 9.403043412539372
mean_absolute_percentage_error on Training set(XGBoost) 7.372936932622327


In [392]:
R2cv1=r2_score(y_test,np.exp(y_pred))
R2t1=r2_score(y_train, predictions_rf)
R2cv2=r2_score(y_test, predictions)
R2t2=r2_score(y_train, predictions1)
print("R2 square on CrossValidation set(Random Forest)",R2cv1)
print("R2 square on Training set(Random Forest)",R2t1)
print("R2 square on CrossValidation set(XGBoost)",R2cv2)
print("R2 square on Training set(XGBoost)",R2t2)

R2 square on CrossValidation set(Random Forest) 0.7757258715062711
R2 square on Training set(Random Forest) 0.9442085994618442
R2 square on CrossValidation set(XGBoost) 0.7530391300954867
R2 square on Training set(XGBoost) 0.8250428647141711


In [396]:
def adj(X,y,r_squared):
    adjusted_r_squared = 1 - (1-r_squared)*(len(y)-1)/(len(y)-X.shape[1]-1)
    return(adjusted_r_squared)

In [403]:
print("adjusted R2 score on CrossValidation set(Random Forest",adj(X_test,y_test,R2cv1))
print("adjusted R2 score on Training set(Random Forest)",adj(X_train,y_train,R2t1))
print("adjusted R2 score on CrossValidation set(XGBoost)",adj(X_test,y_test,R2cv2))
print("adjusted R2 score on Training set(XGBoost)",adj(X_train,y_train,R2t2))

adjusted R2 score on CrossValidation set(Random Forest 0.6598509051178445
adjusted R2 score on Training set(Random Forest) 0.9438194155246968
adjusted R2 score on CrossValidation set(XGBoost) 0.6254426806448216
adjusted R2 score on Training set(XGBoost) 0.8238224166507461



Model Name
R2- Score
Adjusted R2-Score
Mean Absolute Percentage Error (MAPE)
XGBoost Regressor 
(Validation Set)
0.753


0.625


9.403


XGBoost Regressor 
(Training Set)


0.825
0.824


7.373


Random Forest Regression(
Validation Set)
0.776


0.659


8.592


Random Forest Regression(Training Set)
0.944


0.944


3.349
